## Dependencies

## Dependencies

In [1]:
import os
import time
import pandas as pd
import json
import requests
from Alpha_key import API_KEY
from sql_conn import password
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## SQL Stuff

In [3]:
host="secondawsdb.caa0qcwnjnci.us-east-2.rds.amazonaws.com"
port=3307
dbname="stock_data"
user="root"
password=f"{password}"

# host="localhost"
# port=3306
# dbname="stock_data"
# user="root"
# password=f"{password}"

engine = create_engine(f"mysql://root:{password}@{host}:{port}/stock_data")

Base.metadata.create_all(engine)

from sqlalchemy.orm import Session
session = Session(engine)

Base.metadata.tables

immutabledict({})

## Create batches

In [4]:
b1 = ['MSFT', 'WMT', 'V', 'AXP', 'PG']
b2 = ['CVX', 'JNJ', 'HD', 'KO', 'NKE']
b3 = ['JPM', 'MCD', 'UTX', 'MMM', 'AAPL']
b4 = ['GS', 'IBM', 'TRV', 'VZ', 'CSCO']
b5 = ['DIS', 'UNH', 'XOM', 'MRK', 'BA']
b6 = ['INTC', 'DWDP', 'CAT', 'PFE', 'WBA']

DJIA = [b1, b2, b3, b4, b5, b6]

In [8]:
DJIA = ['MSFT', 'WMT', 'V', 'AXP', 'PG',
        'CVX', 'JNJ', 'HD', 'KO', 'NKE',
        'JPM', 'MCD', 'UTX', 'MMM', 'AAPL',
        'GS', 'IBM', 'TRV', 'VZ', 'CSCO',
        'DIS', 'UNH', 'XOM', 'MRK', 'BA',
        'INTC', 'DWDP', 'CAT', 'PFE', 'WBA']

In [29]:
DJIA = ['MSFT', 'WMT', 'V', 'AXP', 'PG']

In [ ]:
Measure = "STOCH"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&series_type={series_type}&apikey={API_KEY}")
STOCH = []
batchNumber = b1

for ticker in batchNumber :
    response = requests.get(callUrl)
    data = response.json()
    for key in data[f'Technical Analysis: {Measure}']:
        data[f'Technical Analysis: {Measure}'][key].update({'Symbol' :f"{ticker}"})
    STOCH.append(data[f'Technical Analysis: {Measure}'])
    
# STOCH_b1 = []
# for x in range(0,5):
#     STOCH[x] = pd.DataFrame.from_dict(STOCH[x], orient='index')
#     STOCH_b1.append(STOCH[x])

In [ ]:
len(STOCH)

In [ ]:
#b1
STOCH_MSFT = STOCH[0]; STOCH_WMT = STOCH[1]; STOCH_V = STOCH[2]; STOCH_AXP = STOCH[3]; STOCH_PG = STOCH[4]

In [ ]:
#b1
STOCH_MSFT = STOCH_MSFT.reset_index(drop=False)
STOCH_WMT = STOCH_WMT.reset_index(drop=False)
STOCH_V = STOCH_V.reset_index(drop=False)
STOCH_AXP = STOCH_AXP.reset_index(drop=False)
STOCH_PG = STOCH_PG.reset_index(drop=False)

In [ ]:
#b1
STOCH_MSFT.to_sql(name = f'{Measure}_MSFT', con = engine,
        if_exists = 'replace', chunksize = 75)
STOCH_WMT.to_sql(name = f'{Measure}_WMT', con = engine,
        if_exists = 'replace', chunksize = 75)
STOCH_V.to_sql(name = f'{Measure}_V', con = engine,
        if_exists = 'replace', chunksize = 75)
STOCH_AXP.to_sql(name = f'{Measure}_AXP', con = engine,
        if_exists = 'replace', chunksize = 75)
STOCH_PG.to_sql(name = f'{Measure}_PG', con = engine,
        if_exists = 'replace', chunksize = 75)

In [62]:
b1 = ['MSFT', 'WMT', 'V', 'AXP', 'PG']
b2 = ['CVX', 'JNJ', 'HD', 'KO', 'NKE']
b3 = ['JPM', 'MCD', 'UTX', 'MMM', 'AAPL']
b4 = ['GS', 'IBM', 'TRV', 'VZ', 'CSCO']
b5 = ['DIS', 'UNH', 'XOM', 'MRK', 'BA']
b6 = ['INTC', 'DWDP', 'CAT', 'PFE', 'WBA']

DJIA = [b1, b2, b3, b4, b5, b6]

## Define Function

In [5]:
def getData():
    #output_df = []
    response = requests.get(f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
    data = response.json()
    for key in data[f'Technical Analysis: {Measure}']:
        data[f'Technical Analysis: {Measure}'][key].update({'Symbol' :f"{ticker}"})
    output_dict = (data[f'Technical Analysis: {Measure}'])
    output_df = pd.DataFrame.from_dict(output_dict, orient='index')
    
    output_df = output_df.reset_index(drop=False).rename(columns = {"index": "date_key"})

    output_df.to_sql(name = f'{Measure.lower()}_{ticker.lower()}', con = engine,
        if_exists = 'replace', chunksize = 75)
#     sql_1 = (
#         f'SELECT * FROM stock_data.{ticker.lower()} '
#         f'INNER JOIN stock_data.{Measure.lower()}_{ticker.lower()} ON '
#         f'stock_data.{ticker.lower()}.Date=stock_data.{Measure.lower()}_{ticker.lower()}.date_key;')
#     sql_2 = (f'ALTER TABLE {ticker.lower()} DROP date_key, DROP Symbol;')
#     print(sql_1, sql_2)
#     engine.execute(sql_1)
#     engine.execute(sql_2)

## First Batch

In [7]:
#b1
Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
#callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
#output1_df = []


for ticker in b6:
    getData()

#output_df = []
# RSI_b1 = []
# for x in range(0,5):
#     output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
#     RSI_b1.append(output_df[x])

NameError: name 'callUrl' is not defined

In [35]:
#b1
RSI = RSI_b1
RSI_MSFT = RSI[0]; RSI_WMT = RSI[1]; RSI_V = RSI[2]; RSI_AXP = RSI[3]; RSI_PG = RSI[4]; 

#b1
RSI_MSFT = RSI_MSFT.reset_index(drop=False)
RSI_WMT = RSI_WMT.reset_index(drop=False)
RSI_V = RSI_V.reset_index(drop=False)
RSI_AXP = RSI_AXP.reset_index(drop=False)
RSI_PG = RSI_PG.reset_index(drop=False)

In [36]:
#b1
RSI_WMT.to_sql(name = f'{Measure}_{ticker}', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_WMT.to_sql(name = f'{Measure}_WMT', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_V.to_sql(name = f'{Measure}_V', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_AXP.to_sql(name = f'{Measure}_AXP', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_PG.to_sql(name = f'{Measure}_PG', con = engine,
        if_exists = 'replace', chunksize = 75)

C:\Users\cwesson\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'RSI_WMT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [38]:
# SQL command to create a table in the database 
sql_command = """SELECT * FROM stock_data.wmt LEFT JOIN stock_data.rsi_wmt ON wmt.Date=rsi_wmt.index""" 
# staff_number INTEGER PRIMARY KEY,  
# fname VARCHAR(20),  
# lname VARCHAR(30),  
# gender CHAR(1),  
# joining DATE);"""
engine.execute(sql_command)

In [36]:
engine.execute(sql_command)

## Second Batch

In [32]:
#b2
Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
#callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
output_df = []


for ticker in b2:
    getData()

RSI_b2 = []
for x in range(0,5):
    output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
    RSI_b2.append(output_df[x])

In [33]:
RSI = RSI_b2

RSI_CVX = RSI[0]
RSI_JNJ = RSI[1]
RSI_HD = RSI[2]
RSI_KO = RSI[3]
RSI_NKE = RSI[4]

RSI_CVX = RSI_CVX.reset_index(drop=False)
RSI_JNJ = RSI_JNJ.reset_index(drop=False)
RSI_HD = RSI_HD.reset_index(drop=False)
RSI_KO = RSI_KO.reset_index(drop=False)
RSI_NKE = RSI_NKE.reset_index(drop=False)

RSI_CVX.to_sql(name = f'{Measure}_CVX', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_JNJ.to_sql(name = f'{Measure}_JNJ', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_HD.to_sql(name = f'{Measure}_HD', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_KO.to_sql(name = f'{Measure}_KO', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_NKE.to_sql(name = f'{Measure}_NKE', con = engine,
        if_exists = 'replace', chunksize = 75)

C:\Users\cwesson\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'RSI_CVX' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\Users\cwesson\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'RSI_JNJ' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\Users\cwesson\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'RSI_HD' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\Users\cwesson\AppData\Local\C

## Third Batch

In [44]:
#b3
Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
#callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
output_df = []


for ticker in b3:
    getData()

RSI_b3 = []
for x in range(0,5):
    output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
    RSI_b3.append(output_df[x])

In [45]:
RSI = RSI_b3

RSI_JPM = RSI[0]
RSI_MCD = RSI[1]
RSI_UTX = RSI[2]
RSI_MMM = RSI[3]
RSI_AAPL = RSI[4]

RSI_JPM = RSI_JPM.reset_index(drop=False)
RSI_MCD = RSI_MCD.reset_index(drop=False)
RSI_UTX = RSI_UTX.reset_index(drop=False)
RSI_MMM = RSI_MMM.reset_index(drop=False)
RSI_AAPL = RSI_AAPL.reset_index(drop=False)

RSI_JPM.to_sql(name = f'{Measure}_JPM', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_MCD.to_sql(name = f'{Measure}_MCD', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_UTX.to_sql(name = f'{Measure}_UTX', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_MMM.to_sql(name = f'{Measure}_MMM', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_AAPL.to_sql(name = f'{Measure}_AAPL', con = engine,
        if_exists = 'replace', chunksize = 75)

## Fourth Batch

In [14]:
#b4 = ['GS', 'IBM', 'TRV', 'VZ', 'CSCO']
def getData():    
    response = requests.get(callUrl)
    data = response.json()
    for key in data[f'Technical Analysis: {Measure}']:
        data[f'Technical Analysis: {Measure}'][key].update({'Symbol' :f"{ticker}"})
    output_df.append(data[f'Technical Analysis: {Measure}'])

Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
output_df = []
for ticker in b4:
    getData()

RSI_b4 = []
for x in range(0,5):
    output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
    RSI_b4.append(output_df[x])

In [15]:
RSI = RSI_b4

RSI_GS = RSI[0]
RSI_IBM = RSI[1]
RSI_TRV = RSI[2]
RSI_VZ = RSI[3]
RSI_CSCO = RSI[4]

RSI_GS = RSI_GS.reset_index(drop=False)
RSI_IBM = RSI_IBM.reset_index(drop=False)
RSI_TRV = RSI_TRV.reset_index(drop=False)
RSI_VZ = RSI_VZ.reset_index(drop=False)
RSI_CSCO = RSI_CSCO.reset_index(drop=False)

RSI_GS.to_sql(name = f'{Measure}_GS', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_IBM.to_sql(name = f'{Measure}_IBM', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_TRV.to_sql(name = f'{Measure}_TRV', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_VZ.to_sql(name = f'{Measure}_VZ', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_CSCO.to_sql(name = f'{Measure}_CSCO', con = engine,
        if_exists = 'replace', chunksize = 75)

## Fifth Batch

In [16]:
#b5 = ['DIS', 'UNH', 'XOM', 'MRK', 'BA']
def getData():    
    response = requests.get(callUrl)
    data = response.json()
    for key in data[f'Technical Analysis: {Measure}']:
        data[f'Technical Analysis: {Measure}'][key].update({'Symbol' :f"{ticker}"})
    output_df.append(data[f'Technical Analysis: {Measure}'])

Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
output_df = []
for ticker in b5:
    getData()

RSI_b5 = []
for x in range(0,5):
    output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
    RSI_b5.append(output_df[x])

In [17]:
RSI = RSI_b5

RSI_DIS = RSI[0]
RSI_UNH = RSI[1]
RSI_XOM = RSI[2]
RSI_MRK = RSI[3]
RSI_BA = RSI[4]

RSI_DIS = RSI_DIS.reset_index(drop=False)
RSI_UNH = RSI_UNH.reset_index(drop=False)
RSI_XOM = RSI_XOM.reset_index(drop=False)
RSI_MRK = RSI_MRK.reset_index(drop=False)
RSI_BA = RSI_BA.reset_index(drop=False)

RSI_DIS.to_sql(name = f'{Measure}_DIS', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_UNH.to_sql(name = f'{Measure}_UNH', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_XOM.to_sql(name = f'{Measure}_XOM', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_MRK.to_sql(name = f'{Measure}_MRK', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_BA.to_sql(name = f'{Measure}_BA', con = engine,
        if_exists = 'replace', chunksize = 75)

## Sixth Batch

In [6]:
# b6 = ['INTC', 'DWDP', 'CAT', 'PFE', 'WBA']
def getData():    
    response = requests.get(callUrl)
    data = response.json()
    for key in data[f'Technical Analysis: {Measure}']:
        data[f'Technical Analysis: {Measure}'][key].update({'Symbol' :f"{ticker}"})
    output_df.append(data[f'Technical Analysis: {Measure}'])

Measure = "RSI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
callUrl = (f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}") + (f"&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
time_period = "20"
output_df = []
for ticker in b6:
    getData()

RSI_b6 = []
for x in range(0,5):
    output_df[x] = pd.DataFrame.from_dict(output_df[x], orient='index')
    RSI_b6.append(output_df[x])

NameError: name 'ticker' is not defined

In [19]:
RSI = RSI_b6

RSI_INTC = RSI[0]
RSI_DWDP = RSI[1]
RSI_CAT = RSI[2]
RSI_PFE = RSI[3]
RSI_WBA = RSI[4]

RSI_INTC = RSI_INTC.reset_index(drop=False)
RSI_DWDP = RSI_DWDP.reset_index(drop=False)
RSI_CAT = RSI_CAT.reset_index(drop=False)
RSI_PFE = RSI_PFE.reset_index(drop=False)
RSI_WBA = RSI_WBA.reset_index(drop=False)

RSI_INTC.to_sql(name = f'{Measure}_INTC', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_DWDP.to_sql(name = f'{Measure}_DWDP', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_CAT.to_sql(name = f'{Measure}_CAT', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_PFE.to_sql(name = f'{Measure}_PFE', con = engine,
        if_exists = 'replace', chunksize = 75)
RSI_WBA.to_sql(name = f'{Measure}_WBA', con = engine,
        if_exists = 'replace', chunksize = 75)

In [35]:
# SQL command to create a table in the database 
sql_command = """CREATE TABLE emp (  
staff_number INTEGER PRIMARY KEY,  
fname VARCHAR(20),  
lname VARCHAR(30),  
gender CHAR(1),  
joining DATE);"""

In [36]:
engine.execute(sql_command)

In [ ]:
#b2
RSI_MSFT = RSI[0]
RSI_WMT = RSI[1]
RSI_V = RSI[2]
RSI_AXP = RSI[3]
RSI_PG = RSI[4]

In [ ]:
#b3
RSI_MSFT = RSI[0]
RSI_WMT = RSI[1]
RSI_V = RSI[2]
RSI_AXP = RSI[3]
RSI_PG = RSI[4]

In [ ]:
#b4
RSI_MSFT = RSI[0]
RSI_WMT = RSI[1]
RSI_V = RSI[2]
RSI_AXP = RSI[3]
RSI_PG = RSI[4]

In [ ]:
#b5
RSI_MSFT = RSI[0]
RSI_WMT = RSI[1]
RSI_V = RSI[2]
RSI_AXP = RSI[3]
RSI_PG = RSI[4]

In [ ]:
#b6
RSI_MSFT = RSI[0]
RSI_WMT = RSI[1]
RSI_V = RSI[2]
RSI_AXP = RSI[3]
RSI_PG = RSI[4]

In [ ]:
Measure = "BBANDS"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
time_period = "20"
BBANDS = []

for ticker in b1:
    response = requests.get(
        f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}&time_period={time_period}&series_type={series_type}&nbdevup=3&nbdevdn=3&apikey={API_KEY}")
    data = response.json()
    BBANDS.append(data)
       
BBANDS

In [ ]:
Measure = "EMA"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
series_type = "close"
time_period = "20"
EMA = []

for ticker in b1:
    response = requests.get(
        f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}&time_period={time_period}&series_type={series_type}&apikey={API_KEY}")
    data = response.json()
    EMA.append(data)
       
EMA

In [ ]:
Measure = "ADX"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
time_period = "20"
ADX = []

for ticker in b1:
    response = requests.get(
        f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}&time_period={time_period}&apikey={API_KEY}")
    data = response.json()
    ADX.append(data)
     
ADX

In [ ]:
Measure = "CCI"
output = "full"
MainUrl = "https://www.alphavantage.co/query?"
interval = "daily"
time_period = "20"
CCI = []

for ticker in b1:
    response = requests.get(
        f"{MainUrl}function={Measure}&symbol={ticker}&interval={interval}&time_period={time_period}&apikey={API_KEY}")
    data = response.json()
    CCI.append(data)
     
CCI

In [ ]:
#sql_add_RSI = con.execute()

In [ ]:
cursor.execute("USE stock_data")
daily_adj.to_sql(
    name = 'daily_adj', conn = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `daily_adj` ADD PRIMARY KEY (`index`);')

In [ ]:
cursor.execute("USE stock_data")
RSI.to_sql(
    name = 'RSI', conn = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `RSI` ADD PRIMARY KEY (`index`);')

In [ ]:
engine.execute("USE stock_data")
RSI.to_sql(
    name = 'RSI', con = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `RSI` ADD PRIMARY KEY (`index`);')

In [ ]:
engine.execute("USE stock_data")
BBANDS.to_sql(
    name = 'BBANDS', con = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `BBANDS` ADD PRIMARY KEY (`index`);')

In [ ]:
engine.execute("USE stock_data")
RSI.to_sql(
    name = 'RSI', con = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `RSI` ADD PRIMARY KEY (`index`);')